### Preprocessing

In [ ]:
!pip install sastrawi

     |████████████████████████████████| 209 kB 24.2 MB/s 


In [ ]:
import pandas as pd
import nltk
import re
import string

from string import punctuation
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm, naive_bayes
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, classification_report

In [ ]:
nltk.download('stopwords')
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
bisnis_df = pd.read_csv('/content/drive/MyDrive/DF NLP/bisnis.csv')
bisnis_df['Label'] = 0
bisnis_df.shape

(40, 6)

In [ ]:
lifestyle_df = pd.read_csv('/content/drive/MyDrive/DF NLP/lifestyle.csv')
lifestyle_df['Label'] = 1
lifestyle_df.shape

(40, 6)

In [ ]:
sport_df = pd.read_csv('/content/drive/MyDrive/DF NLP/sport.csv')
sport_df['Label'] = 2
sport_df.shape

(40, 6)

In [ ]:
df = pd.concat([bisnis_df, lifestyle_df, sport_df], ignore_index=True)

In [ ]:
df.drop(['id','url','Unnamed: 0','title'], axis=1, inplace=True)

In [ ]:
#preprocessing function
def preprocessing(sentence):
    
    #lowecasing
    sentence = sentence.lower()
    sentence = re.sub(r"b'\S+", "", sentence)
    sentence = re.sub(r"https\S+", "", sentence)
    sentence = re.sub(r"x\S+", "", sentence)
    sentence = re.sub(r"@\S+", "", sentence)
    
    #remove white spaces
    sentence = sentence.strip()
    
    #tokenization
    words = sentence.split()
    
    #remove punctuation/ special character
    remove_table = str.maketrans("", "", punctuation)
    words = [x.translate(remove_table) for x in words]
    
    #remove nonalphanumeric <=3 chars
    words = [x for x in words if x.isalnum() and len(x) > 3]
    
    # stemming
    words = [stemmer.stem(w) for w in words]
    
    #remove stopwords
    words = [x for x in words if x not in stop_words]
    
    #rejoining the words
    sentence = " ".join(words)
    
    return sentence

In [ ]:
df['content'] = [preprocessing(stc) for stc in df['content']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['Label'], test_size=0.2)

## Count

In [ ]:
cv = CountVectorizer(lowercase = True, stop_words = stop_words,token_pattern="[A-Za-z]+")
X_train_vect = cv.fit_transform(X_train)
X_test_vect = cv.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


### NBC

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_vect, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
predicted2 = clf.predict(X_test_vect)

In [ ]:
print(classification_report(y_test, predicted2))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.88      0.88      0.88         8
           2       1.00      1.00      1.00         7

    accuracy                           0.92        24
   macro avg       0.92      0.92      0.92        24
weighted avg       0.92      0.92      0.92        24



In [ ]:
from sklearn.metrics import roc_auc_score
prob = clf.predict_proba(X_test_vect)
score = roc_auc_score(y_test, prob, multi_class='ovr')
print(score)

0.9799768518518519


In [ ]:
d = ['Aku bermain bola basket']
tes = cv.transform(d)
clf.predict(tes)

array([2])